# Importing libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Settings

In [2]:
classe_info = {
  'Пшеница - 231 тонн':                {'value':0},
  'Не найдено - 10 тонн':              {'value':1},
  'Подсолнечник - 0 тонн':             {'value':2},
  'Картошка - 1 тонн':                   {'value':3},
  'Табак - 0 тонн':                    {'value':4},
  'Сахарная свекла - 0 тонн':          {'value':5},
  'Капуста - 0 тонн':                  {'value':6},
  'Свекла - 0 тонн':                   {'value':7},
  'Моpковь - 0 тонн':                  {'value':8},
  'Лук - 0 тонн':                      {'value':9},
}

classes = {x : y.get('value') for x, y in classe_info.items()}

features = ['red', 'nir', 'swir']
n_features = len(features)

sequence_size = 30

model_dir = './logs'

# Dataset preparation

In [3]:
# Dataset preparation
url = "time_series.csv"
data = pd.read_csv(url)
data = data.drop(['date', 'id'], axis=1)

X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print('data shape = ',data.shape)
print('X_train, y_train shape = ', X_train.shape, y_train.shape)

data shape =  (1265000, 4)
X_train, y_train shape =  (948750, 3) (948750,)


In [4]:
A = data['class'].unique()
print(A)

[ 5 17  1  8  0 11  3 15  7 12  2 10 16  9 14 18  4  6 13 19]


In [5]:
# Dataset preparation
url = "time_series.csv"
data = pd.read_csv(url)
data = data.drop(['date', 'id'], axis=1)

df_filtered = data[(data['class'] < 10) | (data['class'] > 19)]
df_filtered.to_csv('df_filtered.csv', index=False)

In [6]:
data

,red,nir,swir,class
0,0.125,0.263,0.249,5
1,0.149,0.283,0.254,5
2,0.170,0.304,0.284,17
3,0.191,0.319,0.302,17
4,0.213,0.343,0.319,17
...,...,...,...,...
1264995,0.160,0.305,0.263,2
1264996,0.189,0.354,0.298,17
1264997,0.216,0.376,0.305,17
1264998,0.231,0.389,0.332,17


##### Tuning hyperparameters

In [7]:
input_dim = X_train.shape[1]
output_dim = len(np.unique(y_train))
hidden_dim = 64
num_layers = 1
batch_size = 32
learning_rate = 0.001
num_epochs = 1

##### Check for CUDA-enabled GPU

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} for training.")

Using cuda for training.


##### Move data to GPU

In [9]:
X_train = torch.Tensor(X_train).to(device)
y_train = torch.LongTensor(y_train).to(device)

X_test = torch.Tensor(X_test).to(device)
y_test = torch.LongTensor(y_test).to(device)

##### Create Datasets and DataLoaders

In [10]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# LSTM model

In [11]:
class EncoderDecoderLSTM(nn.Module):
  def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
    super(EncoderDecoderLSTM, self).__init__()
    self.hidden_dim = hidden_dim
    self.num_layers = num_layers

    self.encoder_lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
    self.decoder_lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_dim, output_dim)
    self.relu = nn.ReLU()

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

    _, (hn, cn) = self.encoder_lstm(x.unsqueeze(1), (h0, c0))  # Add a dimension for the sequence length
    out, _ = self.decoder_lstm(hn.transpose(0, 1), (hn, cn))  # Transpose hn to match the expected input shape
    out = self.fc(out.squeeze(1))  # Squeeze the sequence length dimension
    out = self.relu(out)  # Apply ReLU activation function to the output of the linear layer

    return out

## Initialize the model

In [12]:
# Initialize the model, loss function and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EncoderDecoderLSTM(input_dim, hidden_dim, num_layers, output_dim).to(device)

### loss function and optimizer

In [13]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train model

In [14]:
# from IPython.display import clear_output
# import matplotlib.pyplot as plt
import time

# Initialize empty lists for storing the loss and time
train_loss = []
train_time = []

# Enable interactive mode in matplotlib
# plt.ion()

# Create plot for training loss
# fig, ax = plt.subplots(figsize=(15, 6))
# ax.set_xlabel("Epoch")
# ax.set_ylabel("Loss")

# Train the model
for epoch in range(num_epochs):
    start_time = time.time()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Compute and store the training loss and time
    train_loss.append(loss.item())
    train_time.append(time.time() - start_time)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Time: {train_time[-1]:.2f}s')

    # Update plot for training loss
#     ax.plot(train_loss, '-o')
#     ax.set_xlim(1, len(train_loss))
#     ax.set_ylim(min(train_loss), max(train_loss))
#     clear_output(wait=True)
#     display(fig)
#     plt.pause(0.01)

# # Show the final plot
# plt.show()

Epoch [1/1], Loss: 1.1852, Time: 40.27s


# Test model

In [15]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0

    for inputs, labels in test_loader:
        # print(inputs)
        # print(labels)

        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total:.2f}%')

# Save the model checkpoint
# torch.save({'model_state_dict': model.state_dict()}, 'model_checkpoint.pt')
torch.save(model,'cnn.pt')

Test Accuracy: 64.86%


# Test model on custom photo

### Get grom 3 tiff(red, nir swir) one array

In [16]:
import rasterio
import numpy as np
import pandas as pd

# Open the TIFF files and extract the data as NumPy arrays
with rasterio.open('./B04.tif') as src:
    red = src.read(1)

with rasterio.open('./B8A.tif') as src:
    nir = src.read(1)

with rasterio.open('./B11.tif') as src:
    swir = src.read(1)

# Combine the NumPy arrays for the different bands into a single NumPy array
data = np.dstack((red, nir, swir))
data[:,:,1].shape

(53, 36)

### Preparation data

In [17]:
float_array = data.astype(np.float32)
# float_array = float_array[:-5400, :-5400]

# input_tensor_test = torch.from_numpy(float_array).float()
input_tensor_test = torch.from_numpy(float_array)

print(input_tensor_test.shape)
tensor_2d = input_tensor_test.reshape(-1, 3)

# Print the shape of the reshaped tensor
print(tensor_2d)
tensor_2d.shape

torch.Size([53, 36, 3])
tensor([[3097., 4222., 4160.],
        [3226., 4250., 4303.],
        [   0.,    0.,    0.],
        ...,
        [   0.,    0.,    0.],
        [   0.,    0.,    0.],
        [   0.,    0.,    0.]])


torch.Size([1908, 3])

##### transform to torch

In [18]:
scaler = MinMaxScaler()
X2 = scaler.fit_transform(tensor_2d)
# Move data to GPU
X2 = torch.Tensor(X2).to(device)
X2.shape

torch.Size([1908, 3])

### Predict data

In [19]:
model.eval()
with torch.no_grad():
    output_tensor = model.forward(X2)

## Preparation predicted data

##### from torch tensor to numpy array

In [20]:
argmax_tensor = torch.argmax(output_tensor, dim=1)
print('argmax_tensor shape = ', argmax_tensor.shape)

tensor_cpu = argmax_tensor.cpu()

argmax_array = tensor_cpu.numpy()

array_2d = argmax_array.reshape(data[:,:,1].shape)

print('array_2d shape = ', array_2d.shape)
print('array_2d', array_2d)


argmax_tensor shape =  torch.Size([1908])
array_2d shape =  (53, 36)
array_2d [[17 17 15 ... 15 15 15]
 [17 17 17 ... 15 15 15]
 [17 17 17 ... 15 15 15]
 ...
 [17 17 17 ... 15 15 15]
 [17 17 17 ... 15 15 15]
 [17 17 17 ... 15 15 15]]
